In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import seaborn as sns
import matplotlib

### Config

In [2]:
folder_path = "speedups.csv"
data_dict = {}

### Load Data

In [3]:
df = pd.read_csv(folder_path, header=None, names=['name', 'value'])

for _, row in df.iterrows():
    match = re.match(r"([^-]+)-([^.]+).raw", row['name'])
    if match:
        category, test = match.groups()
        value = row['value']
        value = value.strip()
        t = datetime.strptime(value,'%Mm%S.%fs') 
        total_seconds = t.minute * 60 + t.second + t.microsecond/1000000

        if test not in data_dict:
            data_dict[test] = {}
        data_dict[test][category] = total_seconds

speedups_table = pd.DataFrame(data_dict).T

In [4]:
### Normalize data with respect to 1 Thread
normalized_speedups = speedups_table.iloc[0].div(speedups_table)

In [5]:
normalized_speedups

,blackscholes,canneal,dedup,ferret,freqmine,radix,vips
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.166829,1.261872,1.643437,1.950816,1.717197,1.735136,1.987131
4,2.637747,2.013243,2.075069,3.348262,3.894535,3.589908,3.753645
8,3.350496,2.607763,1.094854,3.928479,4.806025,5.595506,4.414465


### Plot 2b

In [11]:
# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
markers = ['o', 's', 'p', 'D', 'v', 'd', 'H']
colors = [
    '#0000ff',  # blue
    '#007f00',  # green
    '#ff0000',  # red
    '#00bfbf',  # light blue
    '#bf00bf',  # pink
    '#bfbf00',  # ocra
    '#000000'   # black
]
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
x_pos = [1, 2, 4, 8]

y_labels = ['1', '2', '3', '4', '5', '6']
ax.set_yticks(range(1, 7, 1), y_labels)

ax.set_xticks(x_pos, ['1', '2', '4', '8'])

# Linear speedup line
linear_speedup = ax.plot([1, 6], [1, 6], color='darkgrey', linestyle='--', label='linear speedup')[0]

# Significant speedup line
step_x = [2, 4, 8, 9]
step_y = [1.75, 3, 4, 4]
significant_speedup = plt.step(step_x, step_y, where='post', linestyle='--', color='k', label='significant speedup')[0]

i = 0
for column in normalized_speedups.columns:
    ax.errorbar(list(map(int, normalized_speedups.index)), normalized_speedups[column], marker=markers[i], color=colors[i],
                 markersize=9, markeredgewidth=1.75, markerfacecolor='none', label=column, alpha=0.7, linewidth=2.5)
    i += 1

ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_xlim(0, 8.5)
ax.set_ylim(0, 6.25)
ax.set_xlabel('Number of Threads', fontsize=20)
ax.set_ylabel('Speedup (normalized to 1-thread)', fontsize=20)
ax.set_title('Parallelized Behaviour (1 run)', fontsize=20)

handles, labels = plt.gca().get_legend_handles_labels()
handles.extend([linear_speedup, significant_speedup])
ax.legend(handles=handles[2:], fontsize=13)
ax.grid()
fig.tight_layout()

if False:
    plt.show()
else:
    matplotlib.use("pgf")
    plt.rc("pgf", texsystem="pdflatex", rcfonts=False)
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        # Use LaTeX default serif font.
        "font.serif": [],
    })
    plt.savefig("plots/plot2b_CCA_guidelines.pgf", bbox_inches="tight", backend='pgf')
    print('done')


done
